In [1]:
import pandas as pd
import numpy as np

In [2]:
import dlp
import os
os.chdir('/project/work')

In [3]:
from preprocess import *
from Data_Load import *
from parameters import *

In [35]:
from pycaret.regression import *

## 1. 기본 데이터 
#### (1시간 단위의 교통 카드 이력 데이터, 정류장별 위/경도 좌표 포함)

In [4]:
# 현재 환경에 마운트되어 있는 데이터셋 키값, 경로를 출력한다
for i, dataset_key in enumerate(dlp.dataset_paths.keys()):
    print(" * 데이터셋 KEY : %s\t DATAPATH: %s"%(dataset_key.rjust(10), dlp.dataset_paths[dataset_key]))

 * 데이터셋 KEY :       mybi	 DATAPATH: /datasets/DTS2021000012


In [45]:
base_data.columns

Index(['Unnamed: 0', 'stop_nm', 'latitude', 'longitude', 'stop_id',
       'transdate', 'totalcnt', 'normalcnt', 'studentcnt', 'childcnt'],
      dtype='object')

In [5]:
base_data = pd.read_csv(dlp.dataset_paths['mybi']+"/data/base_data.csv")
target_col = "totalcnt"

In [6]:
base_data.dtypes

Unnamed: 0      int64
stop_nm        object
latitude      float64
longitude     float64
stop_id         int64
transdate      object
totalcnt        int64
normalcnt       int64
studentcnt      int64
childcnt        int64
dtype: object

In [7]:
#날짜 type변환
base_data['transdate'] = pd.to_datetime(base_data['transdate'])

In [8]:
base_year = base_data["transdate"].dt.strftime("%Y")[0]
city = "울산"

In [9]:
path = os.getcwd() + '/data/api_data'

## 2. 외부 데이터 수집

In [10]:
from utils.createFolder import *

In [11]:
createFolder(os.getcwd() + '/data')
createFolder(os.getcwd() + '/data/api_data')

In [12]:
# 휴일 정보 수집
Load_Holiday_Data(holiday_params_dict, save_tf = True, save_path = path)

In [13]:
# 날씨 정보 수집
Load_Weather_Data(weather_params_dict, save_tf = True, save_path = path)

n_rows : 900, total_count : 8064, max_page = 9


In [14]:
# 미세먼지 경보 정보 수집
Load_Particulate_Matter_Data(pm_params_dict, save_tf = True, save_path = path)

n_rows : 1000, total_count : 304, max_page = 1


In [15]:
# 상권 정보 수집
Load_Trading_Data(trading_params_dict,
                  google_key = googleKey,
                  select_region = city,
                  save_tf = True, 
                  save_path = path)

n_rows : 100, total_count : 36517, max_page = 366


In [16]:
# 병원 정보 수집
Load_Hospital_Data(hospital_params_dict, save_tf = True, save_path = path)

n_rows : 1000, total_count : 1406, max_page = 2


In [17]:
# 학교(초중고) 정보 수집
Load_School_Data(school_params_dict, select_region = city, save_tf = True, save_path = path)

n_rows : 1000, total_count : 500, max_page = 1


In [18]:
# 대학교 정보 수집
Load_University_Data(university_params_dict,
                     google_key = googleKey,
                     select_region = city,
                     save_tf = True, save_path = path)

In [29]:
# 행사 정보 수집
Load_Event_Data(event_params_dict,
                start_year = base_year,
                select_region = city,
                save_tf = True,
                save_path = path)

n_rows : 1000, total_count : 6396, max_page = 7


In [20]:
# 축제 정보 수집
Load_Festival_Data(festival_params_dict,
                   start_year = base_year,
                   select_region = city,
                   save_tf = True,
                   save_path = path)

n_rows : 1000, total_count : 1026, max_page = 2


## 3. 외부 데이터 Load

In [22]:
# 휴일 정보 Load
holiday_data = pd.read_csv(path + "/holiday_data.csv")
# 날씨 정보 Load
weather_data = pd.read_csv(path + "/weather_data.csv")
# 미세먼지 경보 정보 Load
pm_data = pd.read_csv(path + "/pm_data.csv")
# 상권 정보 load
trading_area_data = pd.read_csv(path + "/trading_data.csv")
# 병원 정보 Load
hospital_data = pd.read_csv(path + "/hospital_data.csv")
# 초중고등학교 정보 Load
school_data = pd.read_csv(path + "/school_data.csv")
# 대학교 정보 Load
university_data = pd.read_csv(path + "/university_data.csv")
# 행사 정보 Load
event_data = pd.read_csv(path + "/event_data.csv")
# 축제 정보 Load
festival_data = pd.read_csv(path + "/festival_data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
preprocessing = preprocess(data = base_data,
                           date_col = "transdate", 
                           target_cols = target_col, 
                           stop_id_col = "stop_id", 
                           holiday_data = holiday_data,
                           weather_data = weather_data,
                           pm_data = pm_data,
                           trading_area_data = trading_area_data,
                           hospital_data = hospital_data,
                           school_data = school_data,
                           university_data = university_data,
                           event_data = event_data,
                           festival_data = festival_data,
                           num_cores = 2)

/project/work/model/Preprocessing/create_data.py:89: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["weekofyear"] = data[date_col].dt.week
/project/work/model/Preprocessing/create_data.py:89: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["weekofyear"] = data[date_col].dt.week


In [ ]:
ml_data3 = 

In [24]:
ml_data = preprocessing.run()

1. 결측치 Impute ... Finished ((563300, 15))
2. 시계열 변수 생성 : Finished ((563300, 67))
3. 시간적 특성 변수 추가 (특일, 날씨, 미세먼지 경보) ... Finished ((563300, 75))
4. 공간적 특성 정보 추가 (상권정보, 학교정보, 병원정보) ... Finished ((563300, 173))
5. 시공간적 특성 정보 추가 (행사정보, 축제정보) ... Finished ((563300, 175))


In [65]:
list(ml_data.isna().sum())

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2620,
 5240,
 7860,
 10480,
 13100,
 15720,
 18340,
 2620,
 5240,
 7860,
 10480,
 13100,
 15720,
 18340,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 18340,
 36680,
 18340,
 36680,
 18340,
 36680,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 18340,
 36680,
 18340,
 36680,
 18340,
 36680,
 31440,
 31440,
 49780,
 49780,
 68120,
 68120,
 0,
 0,
 0,
 393,
 0,
 484569,
 563300,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [53]:
ml_data2.drop('Unnamed: 0', axis=1).to_csv('/project/work/ml_data.csv', index=False)

In [63]:
ml_data2['totalcnt_bf_1d']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
563295    0.0
563296    0.0
563297    0.0
563298    0.0
563299    0.0
Name: totalcnt_bf_1d, Length: 563300, dtype: float64

In [62]:
list(ml_data2.isnull().sum())

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2620,
 5240,
 7860,
 10480,
 13100,
 15720,
 18340,
 2620,
 5240,
 7860,
 10480,
 13100,
 15720,
 18340,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 18340,
 36680,
 18340,
 36680,
 18340,
 36680,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 2620,
 5240,
 18340,
 36680,
 18340,
 36680,
 18340,
 36680,
 31440,
 31440,
 49780,
 49780,
 68120,
 68120,
 0,
 0,
 0,
 393,
 0,
 484569,
 563300,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [59]:
list(ml_data2.columns)

['stop_id',
 'stop_nm',
 'transdate',
 'Unnamed: 0',
 'totalcnt',
 'normalcnt',
 'studentcnt',
 'childcnt',
 'dayofweek',
 'hour',
 'month',
 'weekofyear',
 'longitude',
 'latitude',
 'totalcnt_bf_1d',
 'totalcnt_bf_2d',
 'totalcnt_bf_3d',
 'totalcnt_bf_4d',
 'totalcnt_bf_5d',
 'totalcnt_bf_6d',
 'totalcnt_bf_7d',
 'totalcnt_bf_1d_total',
 'totalcnt_bf_2d_total',
 'totalcnt_bf_3d_total',
 'totalcnt_bf_4d_total',
 'totalcnt_bf_5d_total',
 'totalcnt_bf_6d_total',
 'totalcnt_bf_7d_total',
 'totalcnt_ma_hour_mean_2d',
 'totalcnt_ma_hour_std_2d',
 'totalcnt_ma_hour_mean_3d',
 'totalcnt_ma_hour_std_3d',
 'totalcnt_ma_hour_mean_4d',
 'totalcnt_ma_hour_std_4d',
 'totalcnt_ma_hour_mean_5d',
 'totalcnt_ma_hour_std_5d',
 'totalcnt_ma_hour_mean_6d',
 'totalcnt_ma_hour_std_6d',
 'totalcnt_ma_hour_week_mean_14d',
 'totalcnt_ma_hour_week_std_14d',
 'totalcnt_ma_hour_week_mean_21d',
 'totalcnt_ma_hour_week_std_21d',
 'totalcnt_ma_hour_week_mean_28d',
 'totalcnt_ma_hour_week_std_28d',
 'totalcnt_ma_dai

In [58]:
list(ml_data2.dtypes)

[dtype('int64'),
 dtype('O'),
 dtype('<M8[ns]'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 d

In [55]:
ml_data2.describe()

,stop_id,Unnamed: 0,totalcnt,normalcnt,studentcnt,childcnt,hour,longitude,latitude,totalcnt_bf_1d,...,hospital_2,hospital_3,hospital_4,school_0,school_1,school_2,university_0,university_1,event_nearby,festival_nearby
count,563300.000000,563300.000000,563300.000000,563300.000000,563300.000000,563300.000000,563300.000000,563300.000000,563300.000000,560680.000000,...,563300.000000,563300.000000,563300.0,563300.000000,563300.0,563300.000000,563300.0,563300.0,563300.0,563300.0
mean,31049.389313,75194.289182,3.202833,2.930618,0.259331,0.012885,13.300000,129.362713,35.529902,3.204912,...,0.114504,6.503817,0.0,0.007634,0.0,0.022901,0.0,0.0,0.0,0.0
std,6941.880903,94532.318062,5.941901,5.421934,1.237121,0.157461,6.148989,0.060893,0.021184,5.944760,...,0.403065,15.090263,0.0,0.087036,0.0,0.149587,0.0,0.0,0.0,0.0
min,19014.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,129.246863,35.477671,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
25%,24229.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.750000,129.310321,35.524465,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
50%,30619.000000,9406.500000,1.000000,1.000000,0.000000,0.000000,13.500000,129.379093,35.535619,1.000000,...,0.000000,1.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
75%,40404.000000,150231.250000,4.000000,4.000000,0.000000,0.000000,18.250000,129.422853,35.540241,4.000000,...,0.000000,7.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
max,40620.000000,291056.000000,98.000000,83.000000,61.000000,22.000000,23.000000,129.433302,35.561309,98.000000,...,2.000000,108.000000,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0


In [30]:
ml_data.head(3)

Unnamed: 0  totalcnt  normalcnt  \
stop_id stop_nm transdate                                              
40403   공업탑     2020-04-08 00:00:00         0.0       0.0        0.0   
                2020-04-08 05:00:00         0.0      10.0       10.0   
                2020-04-08 06:00:00         1.0      35.0       35.0   

                                     studentcnt  childcnt dayofweek  hour  \
stop_id stop_nm transdate                                                   
40403   공업탑     2020-04-08 00:00:00         0.0       0.0         수     0   
                2020-04-08 05:00:00         0.0       0.0         수     5   
                2020-04-08 06:00:00         0.0       0.0         수     6   

                                    month weekofyear   longitude  ...  \
stop_id stop_nm transdate                                         ...   
40403   공업탑     2020-04-08 00:00:00     4         15  129.310321  ...   
                2020-04-08 05:00:00     4         15  129.310321  ...   
                2020-04-08 06:00:00     4         15  129.310321  ...   

                                     hospital_2  hospital_3  hospital_4  \
stop_id stop_nm transdate                                                 
40403   공업탑     2020-04-08 00:00:00           2          23           0   
                2020-04-08 05:00:00           2          23           0   
                2020-04-08 06:00:00           2          23           0   

                                     school_0  school_1  school_2  \
stop_id stop_nm transdate                                           
40403   공업탑     2020-04-08 00:00:00         0         0         0   
                2020-04-08 05:00:00         0         0         0   
                2020-04-08 06:00:00         0         0         0   

                                     university_0  university_1  event_nearby  \
stop_id stop_nm transdate                                                       
40403   공업탑     2020-04-08 00:00:00             0             0             0   
                2020-04-08 05:00:00             0             0             0   
                2020-04-08 06:00:00             0             0             0   

                                     festival_nearby  
stop_id stop_nm transdate                             
40403   공업탑     2020-04-08 00:00:00                0  
                2020-04-08 05:00:00                0  
                2020-04-08 06:00:00                0  

[3 rows x 171 columns]

In [18]:
del base_data

## 4. 모델 학습

In [36]:
categorical_feature = ["dayofweek", "hour", "ntl_holi", "holi", "seq_holi"]

In [37]:
numerical_feature = [col for col in ml_data.columns if col not in categorical_feature + [target_col]]

#### 데이터 Setup

In [38]:
reg_experiment = setup(ml_data, 
                       target = "totalcnt", 
                       categorical_features = categorical_feature,
                       numeric_features = numerical_feature,
                       ignore_features = ["month", "weekofyear", "longitude", "latitude", "studentcnt", "childcnt", "normalcnt"],
                       train_size = 0.7,
                       n_jobs = 14,
                       session_id = 12345,
#                        log_experiment = True,
#                        log_plots = True,
#                        log_profile = True,
#                        log_data = True,
#                        profile = True,
                       silent = True,
                       experiment_name="Passenger_Demand")

,Description,Value
0,session_id,12345
1,Target,totalcnt
2,Original Data,"(563300, 171)"
3,Missing Values,True
4,Numeric Features,158
5,Categorical Features,5
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(394309, 187)"


#### 초도 모델링

In [39]:
%%time
base_models = compare_models(include = ["lightgbm", "rf", "gbr", "lr", "ridge", "lasso", "en", "knn"])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.2240,6.6876,2.5858,0.8112,0.3249,0.6472,572.4460
rf,Random Forest Regressor,1.2121,6.7360,2.5951,0.8099,0.3226,0.6561,2940.1620
gbr,Gradient Boosting Regressor,1.3468,7.3433,2.7096,0.7928,0.3676,0.6991,518.6560
knn,K Neighbors Regressor,1.5154,11.3687,3.3714,0.6792,0.3947,0.7932,317.7480
ridge,Ridge Regression,1.9895,11.9404,3.4552,0.6630,0.6782,0.7460,4.9380
lr,Linear Regression,2.0015,12.0372,3.4691,0.6603,0.6823,0.7476,3.0270
en,Elastic Net,1.9891,12.4882,3.5335,0.6476,0.6664,0.7431,22.1970
lasso,Lasso Regression,2.0004,12.6017,3.5495,0.6444,0.6684,0.7516,27.9240


CPU times: user 1min 13s, sys: 6.07 s, total: 1min 19s
Wall time: 12h 15min 8s


#### 모델 선택 및 생성

In [40]:
%%time
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.2141,6.5551,2.5603,0.8097,0.3250,0.6492
1,1.2028,6.4464,2.5390,0.8131,0.3241,0.6448
2,1.2296,6.7373,2.5956,0.8120,0.3257,0.6500
3,1.2083,6.5638,2.5620,0.8102,0.3245,0.6460
4,1.2306,6.7554,2.5991,0.8143,0.3222,0.6346
5,1.2286,6.7401,2.5962,0.8147,0.3238,0.6392
6,1.2300,6.7051,2.5894,0.8097,0.3273,0.6583
7,1.2384,7.0267,2.6508,0.8039,0.3256,0.6566
8,1.2385,6.8544,2.6181,0.8106,0.3266,0.6443
9,1.2187,6.4917,2.5479,0.8143,0.3241,0.6494


CPU times: user 1min 8s, sys: 1.63 s, total: 1min 10s
Wall time: 1h 24min 24s


In [42]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 293 kB 845 kB/s eta 0:00:01
     |████████████████████████████████| 80 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 4.6 MB/s eta 0:00:011
     |████████████████████████████████| 133 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 4.1 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


#### 모델 튜닝

In [43]:
%%time
tuned_lightgbm_optuna_tpe = tune_model(lightgbm, 
                                       optimize = "RMSE", 
                                       search_library = "optuna", 
                                       search_algorithm = "tpe", 
                                       fold = 10, 
                                       return_tuner = True, 
                                       n_iter = 10)

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:57:46
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,MAE,MSE,RMSE,R2,RMSLE,MAPE


[I 2021-04-20 23:57:46,527] Searching the best hyperparameters using 394309 samples...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py", line 1310, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 7, in <module>
  File "/home/ds_user1/.local/lib/python3.6/site-packages/pycaret/regression.py", line 1102, in tune_model
    **kwargs,
  File "/home/ds_user1/.local/lib/python3.6/site-packages/pycaret/internal/tabular.py", line 4479, in tune_model_supervised
    model_grid.fit(X_train, y_train, groups=groups, **fit_kwargs)
  File "/home/ds_user1/.local/lib/python3.6/site-packages/optuna/integration/sklearn.py", line 876, in fit
    objective, n_jobs=self.n_jobs, n_trials=self.n_trials, timeout=self.timeout
  File "/home/ds_user1/.local/lib/python3.6/site-packages/optuna/study.py", line 409, in optimize
    show_progress_bar=show_progress_bar,
  File "/home/ds_user1/.local/lib/python3.6/site-packages/optuna/_optimize.py", line 120, in _optimize
    None,
  File "/usr/lib/python3.6/concurrent/fu

KeyboardInterrupt: 